# Enumeración

Es la tercera fase del bloque de reconocimiento. Tras haber verificado que equipos y servicios están activos en la red que estemos auditando, en la fase de enumeración conectaremos con estos equipos y servicios que esten activos en la red para, a través de distintas pruebas, conseguir nombres de usuarios, los nombres de los equipos en la red o identificar recursos compartidos (shares).

### Metodología

Se recomienda seguir usando [cherrytree](https://www.giuspen.com/cherrytree/) para ir guardando las evidencias.

1. <a style="color:#000;text-decoration:none;cursor:pointer" href="#intro">Breve introducción informal</a>
<br><br>
2. <a style="color:#000;text-decoration:none;cursor:pointer" href="#smtp">Enumerar usuarios mediante SMTP</a>
  * Breve introducción informal al protocolo SMTP
  * Enumeración directa a través del protocolo SMTP
    * RCPT TO
    * VERFY
    * EXPN
  * Enumeración con el script smtp-user-enum
  * Enumeración con metasploit
  * Enumeración con patator
<br><br>
3. <a style="color:#000;text-decoration:none;cursor:pointer" href="#api_protocols">NetBIOS, SMB y Samba</a>
  * Introducción informal al concepto de API
  * Breve introducción informal a NetBIOS
  * Breve introducción informal a SMB y su relación con NetBIOS
  * Breve introducción informal a Samba
  * Usando nbtscan para enumerar los equipos usando el servicio NBNS de NetBIOS a través de SMB y Samba
  * Usando scripts de nmap para enumerar a través de los protocolos SMB y Samba
    * smb-os-discovery.nse
    * smb-enum-shares.nse
    * smb-enum-users.nse
  * Usando el cliente de SMB smbclient para enumerar shares
  * Usando smbclient para acceder de forma anónima a un servicio SMB
  * Usando enum4linux para enumerar SMB y Samba
  * Enumerando desde windows
    * net view
    * ntbstat
<br><br>
4. <a style="color:#000;text-decoration:none;cursor:pointer" href="#rpc">RPC (Remote Procedure Call)</a>
  * Breve introducción informal a RPC
  * Enumerando a través de msrpc (Microsoft RPC) con rcpclient desde Kali Linux
<br><br>
5. <a style="color:#000;text-decoration:none;cursor:pointer" href="#snmp">Enumerando cuentas de usuario y dispositivos en la red usando SNMP</a>
  * Breve introducción informal al protocolo SNMP
  * Conseguir las community strings usando onesixtyone y enumerando con snmpwalk
  * Enumerando con metasploit
<br><br>
6. <a style="color:#000;text-decoration:none;cursor:pointer" href="#ntp">NTP (Network Time Protocol)</a>
  * Breve introducción informal al protocolo NTP
  * Consultando información con ntpq
  * Consultando información con nmap
<br><br>
7. <a style="color:#000;text-decoration:none;cursor:pointer" href="#dns">DNS - Transferencias de Zona</a>
  * Breve recordatorio informal del protocolo DNS
  * Breve introducción informal al concepto de zona en el protocolo DNS
  * Breve introducción informal a las transferencias de zona
  * Transferencia de zona con nslookup
  * Transferencia de zona con dig
<br><br>

### Referencias usadas

1. Libros
  * [Practical Packet Analysis, 3rd Edition](https://www.nostarch.com/packetanalysis3)
2. Páginas web
  * [RFC 2821 - SMTP](https://tools.ietf.org/html/rfc2821)
  * [Capturas de tráfico de red](https://github.com/chrissanders/packets)
  * [OpenSSL s_client documentation](http://openssl.cs.utah.edu/docs/apps/s_client.html)
  * [ZoneTransfer.me](https://digi.ninja/projects/zonetransferme.php)
  * [NetBIOS Name Services](https://wiki.wireshark.org/NetBIOS/NBNS)
  * [What is the difference between NetBIOS and SMB?](https://www.quora.com/What-is-the-difference-between-NetBIOS-and-SMB)
  * [Microsoft SMB Protocol and CIFS Protocol Overview](https://msdn.microsoft.com/en-us/library/windows/desktop/aa365233.aspx)
  * [Metasploit Unleashed - SNMP Auxiliary Module for Metasploit](https://www.offensive-security.com/metasploit-unleashed/snmp-scan/)

<a name="intro"></a>

### 1. Breve introducción informal

En la fase de enumeración vamos a aprovecharnos de cómo estan diseñados algunos los protocolos de internet. Cuanto mejor conozcamos los [RFC](https://es.wikipedia.org/wiki/Request_for_Comments) que definen los protocolos, mejor podremos ver qué aspectos de su funcionamiento pueden jugar a nuestro favor y qué fallos han tenido los fabricantes a la hora de implementarlos. Algunos protocolos que veremos cómo enumerar son:

* SMTP
* SMB y Samba
* SNMP
* NTP
* DNS

<a name="smtp"></a>

### 2. Enumerar usuarios mediante SMTP

#### Breve introducción informal al protocolo SMTP

El objetivo del protocolo Simple Mail Transfer Protocol (SMTP) es enviar correos electronicos de manera fiable y eficiente. La especificación del protocolo puede encontrarse en el [RFC 2821](https://tools.ietf.org/html/rfc2821). Normalmente usa el puerto 25 del protocolo TCP si la comunicación no esta cifrada y los puertos 465 y 587 del protocolo TCP cuando la comunicación está cifrada. Grosso modo, veremos qué ocurre cuando se manda un correo electrónico y mandaremos un correo electrónico usando el protocolo SMTP. Atendiendo a que esta es una introducción informal, como todos los protocolos, SMTP tiene una serie palabras (instrucciones o comandos) que podemos utilizar para comunicarnos con otra máquina que entienda el protocolo SMTP. Es como los idiomas, uso palabras en español para comunicarme con otra persona que entienda español, porque si utilizo palabras en español con una persona que no conoce el idioma, no me va a poder entender. Lo mismo ocurre con los protocolos de internet, una máquina que habla el protocolo SMTP se comunica con otra que entiende ese idioma (protocolo). ¿Cuales son estas máquinas que entienden SMTP? Los servidores de correo. Esos que encontramos en la fase de recopilación de información a haciendo consultas a los servidores DNS. Los servidores de correo son los registros MX y el servidor o servidores que están autorizados a enviar correos electrónicos son los registros SPF. Veamos primero una comunicación usando el protocolo SMTP y después enviaremos un correo usando el protocolo directamente. Para ver como es una comunicación a través del protocolo SMTP, podemos descargar una captura del tráfico de red y despues visualizarla en Wireshark. [Chris Sanders](http://chrissanders.org/about/), autor del libro [Practical Packet Analysis, 3rd Edition](https://www.nostarch.com/packetanalysis3), tiene un repositorio con multiples capturas que podemos utilizar para aprender los protocolos de internet. Descargamos la [captura](https://github.com/chrissanders/packets/blob/master/mail_sender_server_2.pcapng?raw=true) correspondiente al envío de email y la guardamos en un fichero que se llame *envio_email.pcapng*:

<img src="img/smtp_pcap_file.png" style="float:left">
<div style="clear:both"></div><br>

Para visualizar el archivo, abrimos el capturador de paquetes, wireshark, en Kali Linux:

<img src="img/smtp_wireshark_1.png" style="float:left">
<div style="clear:both"></div><br>

Una vez abierto wireshark, hacemos click en la opción *Open* del menú *File*:

<img src="img/smtp_wireshark_2.png" style="float:left">
<div style="clear:both"></div><br>

Seleccionamos la captura que acabamos de descargar y hacemos click en el botón *Open*:

<img src="img/smtp_wireshark_3.png" style="float:left">
<div style="clear:both"></div><br>

Tras abrir la captura, podemos ver todos los paquetes que se han enviado y recibido durante la captura de red. Con el fin de filtrar la captura y mostrar la información de un modo más amigable, podemos ir a la opción *Follow* menu *Analyze* y seleccionar *TCP Stream*:

<img src="img/smtp_wireshark_4.png" style="float:left">
<div style="clear:both"></div><br>

Ahora podemos ver los comandos que se han usado para la enviar un correo electrónico usando el protocolo SMTP. En la siguiente captura de pantalla están comentados algunos de ellos:

<img src="img/smtp_wireshark_5.png" style="float:left">
<div style="clear:both"></div><br>

Ahora que hemos visto los comandos, usemoslos para enviar un correo electrónico usando el protocolo SMTP. En este caso, volvemos a usar gmail para enviar el correo. Creamos una cuenta para hacer pruebas en gmail y mandamos el email desde esa cuenta:

<img src="img/smtp_envio_1.png" style="float:left">
<div style="clear:both"></div><br>

Debido a que gmail cifra la comunicación, usaremos el programa [s_client](https://www.openssl.org/docs/manmaster/man1/s_client.html) de la suite openssl para conectarnos a los servidores de gmail usando el puerto 587. El programa *s_client* es un programa cliente que nos permite establecer comunicaciones cifradas, a través de distintos protocolos como HTTP o SMTP, con otros equipos remotos que utilicen el protocolo [TLS/SSL](https://es.wikipedia.org/wiki/Transport_Layer_Security) para cifrar las comunicaciones. Usaremos el programa *s_client* de la suite openssl con las siguientes opciones:

  * **-connect {hostname:puerto}** => indica a qué equipo y a qué puerto queremos conectarnos
  * **-starttls {protocolo}** => usa [TLS](https://es.wikipedia.org/wiki/Transport_Layer_Security) para cifrar la comunicación. en este caso indicaremos el protocolo smtp 
  * **-ign_eof** => Ignorará ciertos carácteres que son sinónimos de fin de documento
  * **-crlf** => Convertirá los carácteres *CR* en *CRLF*
  
**NOTA**
En general, en los sistemas operativos GNU/Linux, cuando pulsamos intro para saltar de línea, se usa el carácter *CR* (Carriage Return) para representar ese cambio de línea. Sin embargo, en los sistemas operativos windows y, como podemos ver en la especificación, tambien el protocolo SMTP, usa los carácteres *CR* y *LF* (Line Feed):

<img src="img/smtp_crlf.png" style="float:left">
<div style="clear:both"></div><br>

Como la distribución Kali Linux es un sistema operativo basada en GNU/Linux, le pasamos la opción -crlf para convertir todos los carácteres *CR* a *CRLF*.
  
Una vez vistas las opciones, usamos el programa *s_client* para conectarnos a gmail:

*openssl s_client -starttls smtp -crlf -ign_eof -connect smtp.gmail.com:587*

<img src="img/smtp_envio_2.png" style="float:left">
<div style="clear:both"></div><br>

Tras iniciar la comunicación, nos mostrará un código 250 y se quedará en espera para que nosotros iniciemos la conversación:

<img src="img/smtp_envio_3.png" style="float:left">
<div style="clear:both"></div><br>

Comenzamos saludando al servidor de correo con el comando *EHLO*:

<img src="img/smtp_envio_4.png" style="float:left">
<div style="clear:both"></div><br>

Tras saludarlo, nos mostrará los comandos disponibles. En este caso, tendremos que autenticarnos ante el servidor con nuestro usuario (email) y contraseña. El método de autenticación que usaremos sera [autenticación básica](https://es.wikipedia.org/wiki/Autenticación_de_acceso_básica). Para ello, primero tenemos que pasar nuestro nombre usuario y contraseña al [sistema de numeración](https://es.wikipedia.org/wiki/Sistema_de_numeración) con base 64. Para ello, podemos usar muchas páginas web que podemos encontrar a través de los buscadores de internet o, por ejemplo, usando un lenguaje de programación como [python](https://www.python.org/). El formato en que debemos codificar, pasar de texto a base64, el usuario y la contraseña es:

**\0**miemail@gmail.com**\0**mipassword

*El carácter \0 indica que finaliza una cadena texto y debe ir al principio y entre medias del email y la constraseña*

Una vez que lo tenemos en ese formato, usamos python para convertirlo a base 64:

<img src="img/smtp_envio_5.png" style="float:left">
<div style="clear:both"></div><br>

Copiamos la cadena de texto en base 64 y utilizamos el comando *AUTH PLAIN* para autenticarnos contra el servidor:

<img src="img/smtp_envio_6.png" style="float:left">
<div style="clear:both"></div><br>

La primera vez que intentemos autenticarnos, el servidor de gmail nos mostrará el siguiente mensaje:

<img src="img/smtp_envio_7.png" style="float:left">
<div style="clear:both"></div><br>

y nos enviará un email a nuestra bandeja de entrada para informarnos que se ha intentado acceder desde una aplicación insegura:

<img src="img/smtp_envio_8.png" style="float:left">
<div style="clear:both"></div><br>

Abrimos el correo y hacemos click en el enlace *permitiendo el acceso a aplicaciones menos seguras* (**Estas pruebas deben realizarse sobre una cuenta de gmail creada para el proposito de aprender el protocolo SMTP, nunca desde nuestra cuenta)**:

<img src="img/smtp_envio_9.png" style="float:left">
<div style="clear:both"></div><br>

Tras hacer click en el enlace veremos que, por defecto, **no** esta permitido el acceso:

<img src="img/smtp_envio_10.png" style="float:left">
<div style="clear:both"></div><br>

Lo habilitamos haciendo click en el botón:

<img src="img/smtp_envio_11.png" style="float:left">
<div style="clear:both"></div><br>

y nos autenticamos de nuevo:

<img src="img/smtp_envio_12.png" style="float:left">
<div style="clear:both"></div><br>

ahora si podemos autenticarnos correctamente. El servidor nos muestra el mensaje *Accepted* para indicarnos que la autenticación ha sido correcta y queda a la espera del siguiente comando:

<img src="img/smtp_envio_13.png" style="float:left">
<div style="clear:both"></div><br>

El primer comando que tenemos que usaremos es *MAIL FROM* para indicar quién envía el email. Aquí introducimos el email con el que nos hemos autenticado:

<img src="img/smtp_envio_14.png" style="float:left">
<div style="clear:both"></div><br>

El siguiente comando *RCPT TO* es para indicar el destinatario del correo. Por ejemplo, para poder comprobar que se recibe el mensaje nos lo podemos enviar a nuestro correo personal:

<img src="img/smtp_envio_15.png" style="float:left">
<div style="clear:both"></div><br>

A continuación, usaremos el comando *DATA* para escribir el cuerpo del email:

<img src="img/smtp_envio_16.png" style="float:left">
<div style="clear:both"></div><br>

Escribimos el contenido que queramos:

<img src="img/smtp_envio_17.png" style="float:left">
<div style="clear:both"></div><br>

y cuando hayamos terminado de escribir, lo indicamos con un punto:

<img src="img/smtp_envio_18.png" style="float:left">
<div style="clear:both"></div><br>

y pulsamos intro para enviar el correo electrónico. El servidor nos indicará con un código 250 que se ha enviado correctamente:

<img src="img/smtp_envio_19.png" style="float:left">
<div style="clear:both"></div><br>

Podemos verificar que lo hemos recibido en la bandeja de entrada del correo al que lo hayamos enviado:

<img src="img/smtp_envio_20.png" style="float:left">
<div style="clear:both"></div><br>

y ver su contenido:

<img src="img/smtp_envio_22.png" style="float:left">
<div style="clear:both"></div><br>

finalmente, nos despedimos del servidor con el comando *quit*:

<img src="img/smtp_envio_21.png" style="float:left">
<div style="clear:both"></div><br>

#### Enumeración directa a través del protocolo SMTP

Para entender que, cualquier programa que utilicemos para enumerar usuarios lo que va a hacer es utilizar el protocolo SMTP, primero enumeraremos usuarios nosotros de forma manual. Para enumerar usuarios usando el protocolo SMTP podemos usar 3 comandos del protocolo:

  * RCPT TO
  * VRFY
  * EXPN
  
Comenzaremos viendo como enumerar usuarios con **RCPT TO**:

El servidor de correo de Google no permite enumerar usuarios, por lo que usaremos la máquina metasploitable 2 que tiene un servicio SMTP vulnerable sin cifrar en el puerto 25:

<img src="img/rcpt_to_1.png" style="float:left">
<div style="clear:both"></div><br>

Como la comunicación no esta cifrada, usaremos el programa telnet para conectar con el servicio SMTP. La forma de utilizarlo es:

**telnet {hostname_o_IP} {puerto}**

En este caso nos conectamos al puerto 25 del protocolo TCP de la máquina metasploitable 2:

<img src="img/rcpt_to_2.png" style="float:left">
<div style="clear:both"></div><br>

Tras establecer la conexión, el servidor se queda esperando a recibir comandos. Comenzamos saludando al servidor con el comando *EHLO*:

<img src="img/rcpt_to_3.png" style="float:left">
<div style="clear:both"></div><br>

Tras saludarlo, nos mostrará la lista de comandos disponibles y, nuevamente, se quedará esperando a recibir otro comando. En este caso vamos a comprobar los usuarios de la máquina. Un usuario que, aunque no deberian, tienen muchas máquinas GNU/Linux es el usuario *root*, luego usamos este usuario en el comando *MAIL FROM*:

<img src="img/rcpt_to_4.png" style="float:left">
<div style="clear:both"></div><br>

ahora usaremos el comando *RCPT TO* para intentar averiguar usuarios. Comenzamos haciendo una prueba e intentamos enviar un email a un usuario de la máquina que no existe:

<img src="img/rcpt_to_5.png" style="float:left">
<div style="clear:both"></div><br>

y como podemos ver, el servidor nos indica que ese usuario no existe. Ahora probamos con uno que si existe:

<img src="img/rcpt_to_6.png" style="float:left">
<div style="clear:both"></div><br>

y el servidor nos permite enviarlo a este destinatario. Ahora probamos con otro a ver si existe:

<img src="img/rcpt_to_7.png" style="float:left">
<div style="clear:both"></div><br>

y como no existe nos lo rechaza. Probamos con otro que si existe:

<img src="img/rcpt_to_8.png" style="float:left">
<div style="clear:both"></div><br>

y nos permite enviar correos a ese usuario. En este caso, hubiesemos encontrado dos usuarios:

  * user
  * msfadmin
  
cuando terminemos cerramos la conexión con el servidor, usando el comando quit:

<img src="img/rcpt_to_9.png" style="float:left">
<div style="clear:both"></div><br>

En muchas empresas, es posible enumerar usuarios usando *RCPT TO* porque, para enviar correos, debe estar activo.

**Enumerando con VRFY**

Aunque debería estar desactivado, el siguiente comando que podemos usar para enumerar usuarios a través del protocolo SMTP es el comando *VRFY*. Para ello nos conectamos al servidor usando telnet y le saludamos usando el comando EHLO:

<img src="img/vrfy_1.png" style="float:left">
<div style="clear:both"></div><br>

El comando VRFY se usa de la siguiente manera:

**VRFY {usuarioQueQueremosProbar}**

Por ejemplo, probamos uno que no existe:

<img src="img/vrfy_2.png" style="float:left">
<div style="clear:both"></div><br>

Ahora probamos con uno que si existe:

<img src="img/vrfy_3.png" style="float:left">
<div style="clear:both"></div><br>

y al igual que con el comando *RCPT TO* iriamos probando usuarios a ver si existen o no. Siempre ayudandonos de todos los correos electronicos e información que hayamos podido recopilar en la fase de recopilación de información.

**Enumerando con EXPN**

Aunque el comando *EXPN* del protocolo SMTP también nos permite enumerar, no tengo ninguna máquina de ejemplo. Pero podéis apuntarlo para que lo tengáis en cuenta.

#### Enumeración con el script smtp-user-enum

Ahora usaremos programas que hacen uso de estos comandos para enumerar usuarios con nosotros. Comenzamos por el script escrito en el lenguaje de programación [perl](https://www.perl.org/) *smtp-user-enum*. Comenzamos creando una lista (también llamada diccionario) de usuarios con el editor de texto que queramos. En este caso uso [vim](http://www.vim.org/):

<img src="img/smtp_user_enum_1.png" style="float:left">
<div style="clear:both"></div><br>

Creo unos pocos usuarios para el ejemplo:

<img src="img/smtp_user_enum_2.png" style="float:left">
<div style="clear:both"></div><br>

Una vez que tengo un fichero (diccionario) con los usuarios a comprobar puedo usar el script. Para usarlo le tengo que indicar qué comando del protocolo SMTP utilizar para la enumeración. Las opciones del comando son:

  * **-U {ficheroUsuarios}** Para indicarle al programa cual es el fichero que contiene los usuarios que quiero comprobar
  * **-M {comandoSMTPAUtilizar}** Para indicarle el comando del protocolo SMTP a utilizar
  * **-t {máquinaObjetivo}** Para indicarle el nombre de dominio o IP de la máquina donde esté el servicio
  
La forma de usarlo, por ejemplo, con el comando *VRFY* es:

*smtp-user-enum -M VRFY -U usuarios_a_comprobar -t 192.168.119.8*

<img src="img/smtp_user_enum_3.png" style="float:left">
<div style="clear:both"></div><br>

Podemos ver que, al finalizar, nos informa de los usuarios que exiten:

  * user
  * msfadmin
  
También podemos ejecutar el script indicandole que la comprobación de usuarios la haga a través del comando *RCPT TO* del protocolo SMTP:

*smtp-user-enum -M RCPT -U usuarios_a_comprobar -t 192.168.119.8*

<img src="img/smtp_user_enum_4.png" style="float:left">
<div style="clear:both"></div><br>

#### Enumeración con metasploit

El framework [Metasploit](https://www.metasploit.com/) también incluye un escáner que nos permite enumerar usuarios usando el protocolo SMTP. Para usarlo, primero iniciamos el servicio de la base de datos, [PostgreSQL](https://www.postgresql.org/), que es la que usa metasploit:

<img src="img/smtp_metasploit_1.png" style="float:left">
<div style="clear:both"></div><br>

Una vez iniciado el servicio, iniciamos metasploit:

<img src="img/smtp_metasploit_2.png" style="float:left">
<div style="clear:both"></div><br>

Seleccionamos el escáner que queremos usar con el comando *use auxiliary/scanner/smtp/smtp_enum*:

<img src="img/smtp_metasploit_3.png" style="float:left">
<div style="clear:both"></div><br>

A continuación usamos el comando *show options* para que nos muestre las opciones que tiene el escaner. En este caso cambiaremos las opciones:

  * **RHOSTS** para indicarle la máquina donde está el servicio SMTP
  * **USER_FILE** para indicarle el fichero (diccionario) con la lista de usuarios a comprobar
  
<img src="img/smtp_metasploit_4.png" style="float:left">
<div style="clear:both"></div><br>

Cambiamos las opciones usando el commando *set*:

<img src="img/smtp_metasploit_5.png" style="float:left">
<div style="clear:both"></div><br>

y ejecutamos el escáner con el comando *exploit*:

<img src="img/smtp_metasploit_6.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que descubre los mismos usuarios:

  * user
  * msfadmin
  
#### Enumeración con patator

[Patator](https://github.com/lanjelot/patator) es una herramienta que podemos utilizar para lanzar ataques de diccionario contra distintos servicios. En este caso la usaremos para lanzar un ataque de diccionario contra el servicio smtp con el fin de enumerar usuarios. Para ver cómo se usa, escribimos *patator* en la consola de comandos y veremos que tiene 2 módulos para enumerar usuarios usando los comandos *VRFY* y *RCPT TO* del protocolo SMTP:

<img src="img/smtp_patator_1.png" style="float:left">
<div style="clear:both"></div><br>

Dependiendo de los módulos que utilicemos, tendremos que pasar unas opciones u otras. Por ejemplo, para saber cómo utilizar el módulo *smtp_vrfy*, escribimos *patator smtp_vrfy* y pulsamos intro:

<img src="img/smtp_patator_2.png" style="float:left">
<div style="clear:both"></div><br>

Nos aparecerá un ejemplo. Siguiendo el ejemplo, lanzamos *patator* pasandole el fichero (diccionario) de *usuarios_a_comprobar* y le decimos el saludo que debe usar con el servidor:

<img src="img/smtp_patator_3.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que encuentra los dos usuarios que son correctos:
 
  * user
  * msfadmin

Con el fin de ver mejorar la visualización de los resultados, usamos la opción *-x ignore:code=550* para ignorar aquellos usuarios que no son válidos:

<img src="img/smtp_patator_4.png" style="float:left">
<div style="clear:both"></div><br>

Usando el módulo *smtp_rcpt*, hubiésemos obtenido el mismo resultado:

<img src="img/smtp_patator_5.png" style="float:left">
<div style="clear:both"></div><br>

<a name="api_protocols"></a>

### 3. NetBIOS, SMB y Samba

#### Introducción informal al concepto de API

Una [API](https://es.wikipedia.org/wiki/Interfaz_de_programación_de_aplicaciones) define el lugar (endpoint) donde tengo que llamar para pedir las cosas que quiero pero no el medio de transporte (protocolo) a través del cuál me llegarán las cosas (recursos) que pido. Extrapolemoslo la definición de API a un ejemplo de la vida cotidiana como pedir una pizza a domicilio. Para pedir una pizza a domicilio tengo que llamar a la pizzeria. En este caso el recurso que solicito en mi llamada es una pizza y el endpoint es el lugar al que llamo para hacer el pedido. Lo único que tengo que saber para pedir una pizza es dónde llamar. No me tengo que preocupar por el medio de transporte. Me da igual si me la traen andando, en bici, en coche o en helicoptero. Mi única preocupación es saber a dónde llamar. En una API ocurre lo mismo, sólo tengo que saber a dónde llamar para que me den las cosas (recursos o servicios) que quiero. No me tengo que preocupar por el medio de transporte (protocolo) a través del cual pido y me traen los recursos.

#### Breve introducción informal a NetBIOS

[NetBIOS](https://es.wikipedia.org/wiki/NetBIOS) (Network Basic Input/Output System) es una [API](https://es.wikipedia.org/wiki/Interfaz_de_programación_de_aplicaciones) que fue diseñada por [IBM](https://es.wikipedia.org/wiki/IBM) y más tarde adoptada por [Microsoft](https://es.wikipedia.org/wiki/Microsoft), para que varios ordenadores pudiesen comunicarse entre si y transferir datos entre ellos. NetBIOS permite que aplicaciones en diferentes ordenadores puedan comunicarse entre sí. Asimismo como hemos visto, una API como la de NetBIOS, sólo define el lugar (endpoint) al que tengo que llamar para pedir las cosas (servicios en este caso) pero no define el medio de transporte (protocolo) a través del cual pediré o me llegarán los datos. Los medios de transporte (protocolos) que usan para las llamar a los servicios de la API de NetBIOS son:

  * **UDP 137** => Se usa para las llamadas al servicio de nombres, conocido como **NBNS (NetBIOS Name Service) o WINS**, de la API de NetBIOS
  * **UDP 138** => Se usa para las llamadas al servicio, **Datagram distribution service**, de la API de NetBIOS (en deshuso)
  * **TCP 139** => Se usa para las llamadas al servicio **NetBIOS Session Services** de la API de NetBIOS
  
Estos servicios de la API de NetBIOS son usados por otros protocolos como el protocolo SMB.

**NetBIOS Name Service (NBNS)**

El servicio NetBIOS Name Service (NBNS), también conocido como WINS, es un servicio de la capa 5 (session) del modelo [OSI](https://en.wikipedia.org/wiki/OSI_model) que asigna un nombre a cada ordenador con sistema operativo windows. Sirve al mismo propósito que el protocolo DNS, traducir nombres fáciles de recordar por las personas a direcciones IP. Sólo que NetBIOS, además de funcionar con el protocolo IP, funciona con otros protocolos ya en deshuso como IPX o Token Ring. Sin embargo, el servicio NBNS de la API de NetBIOS está mucho más limitado que el servicio DNS porque, por ejemplo, ni soporta el protocolo IPv6 ni está organizado de forma jerárquica. El nombre que asigna el servicio NBNS esta formado por 16 carácteres [ASCII](https://es.wikipedia.org/wiki/ASCII):

  * los 15 primeros carácteres se usan para el dar el nombre que identifique al equipo o dispositivo
  * el último carácter (sufijo) se utiliza para identificar el tipo de servicio al que se le ha asignado el nombre
  
<img src="img/netbios_ns_suffixes.png" style="float:left">
<div style="clear:both"></div><br>

#### Breve introducción informal a SMB y su relación con NetBIOS

Server Message Block (SMB) es un protocolo que, entre otros, permite el acceso compartido a archivos, impresoras o recursos compartidos (shares) o cuentas de acceso entre equipos con sistema operativo windows. No sólo funciona en una red de área local sino que es accesible desde internet. SMB puede usarse de formas:

  * directamente a traves del puerto 445 del protocolo TCP
  * usando los servicios de la API de NetBIOS a través de:
    * los puertos 137 y 138 del protocolo UDP
    * el puerto 139 del protocolo TCP. 

En los sistemas windows actuales, normalmente, el protocolo SMB se usará en el puerto 445 del protocolo TCP. Sin embargo, SMB usa los servicios de la API de NetBIOS para poder comunicarse algunos servicios SMB que no soportan la conexión directa a traves del protocolo 445 del protocolo TCP.

#### Breve introducción informal a Samba

[Samba](https://www.samba.org/) es una implementación de software libre del protocolo SMB que se usa en sistemas operativos GNU/Linux y que es compatible con equipos con sistema operativo windows.

#### Usando nbtscan para enumerar los equipos usando el servicio NBNS de NetBIOS a través de SMB y Samba

Vamos a comenzar enumerando los nombres de los equipos asignados por el servicio NBNS (NetBIOS Name Service) haciendo llamadas a la API de NetBIOS, a través del puerto 137 del protocolo UDP, desde el protocolo SMB o Samba. Para ello, utilizamos el programa [nbtscan](http://www.unixwiz.net/tools/nbtscan.html). Primero comenzamos enumerando en la red objetivo:

<img src="img/nbtscan_1.png" style="float:left">
<div style="clear:both"></div><br>

y vemos que encontramos dos equipos:

  * IE10WIN7
  * METASPLOITABLE
  
Si los analizamos para que nos muestre los sufijos del nombre asignado por NBNS y asi conocer qué tipo de servicio son, pasamos la opción *-v* a nbtscan:

<img src="img/nbtscan_2.png" style="float:left">
<div style="clear:both"></div><br>

El número de sufijo nos da indicación del uso. Por ejemplo, como hemos visto anteriormente el sujifo *00* nos indica que es una workstation y el sufijo *20* indica un servicio de compartir ficheros.

#### Usando scripts de nmap para enumerar a través de los protocolos SMB y Samba

Estos scripts nos ayudarán en el proceso de enumeración creando conexiones a través del puerto 445 del protocolo TCP con otros servicios SMB o Samba y, también creando conexiones a través del puerto 139 del protocolo TCP usando el servicio de sesiones de la API de NetBIOS. Podemos ver todos los scripts que tenemos disponibles de nmap para usar con el protocolo smb y samba, listando el contenido de la ruta */usr/share/nmap/scripts* y filtrando los resultados con grep:

<img src="img/smb_nmap_scripts.png" style="float:left">
<div style="clear:both"></div><br>

A continuación veremos como usar algunos de ellos.

**smb-os-discovery.nse**

El script intenta determinar el sistema operativo, nombre del ordenador (NBNS), dominio, grupo de trabajo y hora del sistema operativo. Es decir, hace llamadas a los servicios de la API de NetBIOS a través de los puertos 137 (UDP) y 139 (TCP) utilizando el protocolo SMB. Lo probamos en la máquina virtual de Windows 7:

<img src="img/nmap_scripts_1_smb_os_discovery.png" style="float:left">
<div style="clear:both"></div><br>

**smb-enum-shares.nse**

El script intenta enumerar los recursos compartidos (shares) de un sistema utilizando el protocolo SMB. Lo probamos en la máquina de windows 7:

<img src="img/nmap_scripts_5_smb_enum_shares.png" style="float:left">
<div style="clear:both"></div><br>

De todos los recursos compartidos (shares) que se enumeran, el más interesante es el share ** IPC\$** que es conocido como **Null Session**. Si el share **IPC\$** (null session) estuviese activo, nos permitiría el acceso de forma anónima. Es decir, sin necesidad de autenticarnos. Por defecto, en los sistemas windows viene desactivado.

**smb-enum-users.nse**

El script intenta enumerar usuarios utilizando la API de NetBIOS a través del puerto y 139 (TCP) a través del protocolo SMB y utilizando el puerto 445 del protocolo SMB. Debido a que los scripts de nmap intentan utilizar el acceso anónimo (*null session*) y, como hemos visto en el ejemplo anterior, en sistemas operativos windows el acceso anónimo (*null session*) está deshabilitado por defecto, lo probamos en la máquina de metasploitable 2:

<img src="img/nmap_scripts_2_smb_enum_users_1.png" style="float:left">
<div style="clear:both"></div><br>

Como es normal en un sistema GNU/Linux, la mayoría de las cuentas están deshabilitadas. Pero si buscamos, nos encontramos a los 2 mismos usuarios que cuando enumeramos el protocolo SMTP. El usuario msfadmin:

<img src="img/nmap_scripts_3_smb_enum_users_2.png" style="float:left">
<div style="clear:both"></div><br>

y el usuario user:

<img src="img/nmap_scripts_4_smb_enum_users_3.png" style="float:left">
<div style="clear:both"></div><br>

#### Usando el cliente de SMB smbclient para enumerar y conectarnos a shares

También podemos usar un programa cliente del protocolo SMB para enumerar shares (recursos compartidos). Para enumerar shares usamos la opción *-L* Lo probamos contra la máquina metasploitable 2:

<img src="img/smbclient_1.png" style="float:left">
<div style="clear:both"></div><br>

Como está activa la autenticación anónima (*null session*) podemos conectarnos sin necesidad de contraseña. Por eso pulsamos intro:

<img src="img/smbclient_2.png" style="float:left">
<div style="clear:both"></div><br>

En este caso, aparte de la sesión anónima, vemos que hay un share que una carpeta compartida (*/tmp*), pues nos conectamos a ver que tiene usando smbclient:

<img src="img/smbclient_3.png" style="float:left">
<div style="clear:both"></div><br>

Como esta activa la autenticación anónima (*null session*), no necesitamos contraseña:

<img src="img/smbclient_4.png" style="float:left">
<div style="clear:both"></div><br>

Una vez dentro, podemos ver los contenidos de la carpeta usando el comando *ls*:

<img src="img/smbclient_5.png" style="float:left">
<div style="clear:both"></div><br>

#### Usando enum4linux para enumerar SMB y Samba

Otro programa que podemos utilizar para enumerar a través de los protocolos SMB y Samba es [enum4linux](https://tools.kali.org/information-gathering/enum4linux). Como casi todas las herramientas que usamos, *enum4linux* tiene muchas opciones. Vamos a ver dos de ellas. La opción *-U* que nos muestra una lista de usuarios:

<img src="img/enum4linux_1.png" style="float:left">
<div style="clear:both"></div><br>

En este caso vemos que, nos indica que se aceptan sessiones nulas sin usuario ni password, y después nos facilita la lista de usuarios. Para hacer una enumeración completa, usamos la opción *-a*:

<img src="img/enum4linux_2.png" style="float:left">
<div style="clear:both"></div><br>

#### Enumerando desde windows

Desde Windows también tenemos programas que nos van a permitir enumerar a través del protocolo SMB y la API de NetBIOS.

**net view**

Nos puede servir para enumerar los equipos en el dominio y sus recursos compartidos (shares). Para enumerar los equipos del dominio realiza peticiones al servicio NBNS (NetBIOS Name Service) de la API de NetBIOS (puerto 137 del protocolo UDP) a través del protocolo SMB. Por ejemplo, si queremos enumerar los equipos:

<img src="img/net_view_1.png" style="float:left">
<div style="clear:both"></div><br>

Una vez descubierto los equipos, podemos intentar enumerar sus recursos compartidos. Probamos a enumerar los recursos compartidos de la máquina con sistema operativo Windows 10:

<img src="img/net_view_2.png" style="float:left">
<div style="clear:both"></div><br>

**nbtstat**

Una vez descubiertos los equipos con net view, podemos usarlo para ver los sufijos asignados por NBNS:

<img src="img/nbtstat_1.png" style="float:left">
<div style="clear:both"></div><br>

<a name="rpc"></a>

### 4. RPC (Remote Procedure Call)

#### Breve introducción informal a RPC

[RPC](https://es.wikipedia.org/wiki/Llamada_a_procedimiento_remoto) (Remote Procedure Call) es modelo o estandar que se puede utilizar para diseñar una API. Si se diseña una API siguiendo el modelo RPC, la API será una API RPC. Existen otros modelos a la hora de diseñar una API como el modelo [SOAP](https://es.wikipedia.org/wiki/Simple_Object_Access_Protocol) o el modelo [REST](https://en.wikipedia.org/wiki/Representational_state_transfer) que es muy utilizado en APIs web como, por ejemplo, la API REST de Twitter. Entrar en los detalles del diseño de APIs de acuerdo al modelo RPC se salen del objetivo de este documento.

#### Enumerando a través de msrpc (Microsoft RPC) con rcpclient desde Kali Linux

La implementación de RPC de Microsoft se llama MSRPC (Microsoft RPC) que usa como medio de transporte el protocolo SMB. Microsoft RPC lo utilizan muchos servicios de Microsoft Windows Server como el servicio de administración de DNS o Microsoft Exchange. Veamos un ejemplo de cómo podemos utilizar MSRPC para enumerar a través del protocolo SMB en un sistema operativo windows 10. Usaremos el programa **rcpclient** incluido en Kali Linux. Hacemos un escaneo con nmap para ver descubrir el servicio:

<img src="img/rpcclient_1.png" style="float:left">
<div style="clear:both"></div><br>

Para poder enumerar necesitamos tener un usuario y una contraseña. Para usarlo le indicamos el equipo al que nos queremos conectar y el usuario (IEUser en este caso):

<img src="img/rpcclient_2.png" style="float:left">
<div style="clear:both"></div><br>

A continuación, tecleamos la contraseña:

<img src="img/rpcclient_3.png" style="float:left">
<div style="clear:both"></div><br>

Ahora podemos ir tecleando comandos para ir obteniendo información. Por ejemplo *svrinfo* nos mostrará información sobre el sistema operativo:

<img src="img/rpcclient_4.png" style="float:left">
<div style="clear:both"></div><br>

si queremos enumerar los usuarios podemos usar *enumdomusers*:

<img src="img/rpcclient_5.png" style="float:left">
<div style="clear:both"></div><br>

si queremos ver más información sobre algún usuario, podemos usar el comando *queryuser* y pasarle el [RID](https://es.wikipedia.org/wiki/Relative_ID) que nos ha dado en el comando anterior. Podemos pasarle el RID en hexadecimal, o convertirlo en decimal, por ejemplo, usando python:

<img src="img/rpcclient_6.png" style="float:left">
<div style="clear:both"></div><br>

En este caso consultamos información sobre el usuario *Administrator*:

<img src="img/rpcclient_7.png" style="float:left">
<div style="clear:both"></div><br>

Asimismo también podemos enumerar recursos compartidos (shares) utilizando el comando *netshareenumall*:

<img src="img/rpcclient_8.png" style="float:left">
<div style="clear:both"></div><br>

También podemos ver las impresoras configuradas con el comando *enumprinters*:

<img src="img/rpcclient_9.png" style="float:left">
<div style="clear:both"></div><br>

además de enumerar, el programa **rpcclient** nos va a permitir hacer muchas más cosas. Para ver un listado completo de todos los comandos que podemos usar, tecleamos *help*:

<img src="img/rpcclient_10.png" style="float:left">
<div style="clear:both"></div><br>

<a name="snmp"></a>

### 5. Enumerando cuentas de usuario y dispositivos en la red usando SNMP

#### Breve introducción informal al protocolo SNMP

[SNMP](https://es.wikipedia.org/wiki/Simple_Network_Management_Protocol) (Simple Network Management Protocol) es un protocolo que sirve para administrar de forma remota los dispositivos conectados a una red. Usa el puerto 161 del protocolo UDP. De las 3 versiones existentes del protocolo, sólo en una de ellas se cifra la información ([SNMPv3](https://tools.ietf.org/html/rfc3410)) y no es muy utilizada. Desde el punto de vista de la enumeración, nos va a permitir recopilar información de cualquier dispositivo conectado en la red como routers, switches, servidores, impresoras, etc. La mayoría de los fabricantes soportan este protocolo de red que se usa tanto en sistemas operativos Windows como GNU/Linux. Funciona de acuerdo al modelo de [agentes de software](https://es.wikipedia.org/wiki/Agente_%28software%29). Es decir, se instala un programa, que se denomina agente, en cada uno de los dispositivos o equipos que se quiere administrar de forma remota y, se instala un programa, que se denomina manager, en el equipo que se vaya a utilizar para controlar todos los agentes. Los agentes reciben datos e instrucciones del programa que los controla (manager) y envían información al programa manager como su estado actual o su configuración. El programa manager guarda toda esta información en una base de datos que se conoce como [MIB](https://es.wikipedia.org/wiki/Management_Information_Base) (Management Information Base). En esta base de datos (MIB) se ordena la información recopilada de forma jerárquica o siguiendo una estructura de árbol:

<img src="img/snmp_1.png" style="float:left">
<div style="clear:both"></div><br>

Cada punto (nodo) del árbol es una o varias características (opciones) de los dispositvos o equipos que pueden ser administradas remotamente. A cada nodo se le asigna un identificador que se denomina OID (Object Identifier) y que será utilizado por el protocolo SNMP cuando se quiera modificar o saber el estado de esa característica. El protocolo SNMP define 2 contraseñas que denomina **community strings**:

  * una de ellas que sólo permite ver el estado (configuración) del dispositivo y es accesible a todo el mundo (**public community string**)
  * otra que, además de ver, permite cambiar la configuración del dispositivo y que no está disponible públicamente (**private community string**)
  
Como he mencionado anteriormente, de las 3 versiones del protocolo SNMP, la única que va cifrada (SNMPv3) no es muy utilizada. Es decir, que siempre que no este implementada esta versión, podremos, al menos, conseguir la contraseña pública (**public community string**) y ver la configuración de todos los dispositivos. Vamos a ver herramientas (programas) que capturan esta clave pública (public community string) y nos muestran la configuración de los equipos. Como siempre, para ampliar conocimientos podéis consultar los RFCs:

  * [RFC 1157](https://tools.ietf.org/html/rfc1157) => SNMP
  * [RFC 3410](https://tools.ietf.org/html/rfc3410) => SNMPv3
  
#### Conseguir las community strings usando onesixtyone y enumerando con snmpwalk

Antes de comenzar, descubrimos el servicio en el puerto 161 del protocolo UDP usando nmap:

<img src="img/snmp_nmap.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que están usando la primera versión del protocolo SNMP. Después usamos el programa [onesixtyone](https://github.com/trailofbits/onesixtyone) para realizar peticiones SNMP a diferentes IPs para descubrir la contraseña (community string) pública y, a ser posible, también la privada. Por ejemplo, para conseguir las **community strings** de un dispositivo en concreto:

<img src="img/onesixtyone.png" style="float:left">
<div style="clear:both"></div><br>

En este caso han dejado por defecto:

  * **public** para la *public community string*
  * **private** para la *private community string*
  
Ahora podemos usar un programa como snmpwalk para ver la información del equipo de la base de datos MIB ya que tenemos la *public community string* para autorizar esta operación. Para ello, le pasamos las opciones:

  * **-v1** porque la versión del protocolo que hemos descubierto con nmap es *SNMPv1*
  * **-c public** porque estamos usando la *public community string* para autorizar en MIB la operación de ver la configuración del equipo

<img src="img/snmpwalk.png" style="float:left">
<div style="clear:both"></div><br>

#### Enumerando con metasploit

Una vez tenemos la *public community string*, también podemos usar metasploit para ver la información de un dispositivo o equipo. Para ello, usamos el scanner: *auxiliary/scanner/snmp/snmp_enum*

<img src="img/snmp_metasploit_1.png" style="float:left">
<div style="clear:both"></div><br>

Utilizamos el comando *show options* para que nos muestre las opciones del scanner:

<img src="img/snmp_metasploit_2.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que, por defecto, usará:

  * el valor **public** como contraseña (community string)
  * la versión **1** de SNMP para enumerar
  
Como estás opciones son las que correctas en este caso, sólo modificamos la opción *RHOSTS* para indicar el dispositivo o equipo objetivo:

<img src="img/snmp_metasploit_3.png" style="float:left">
<div style="clear:both"></div><br>

Finalmente, ejecutamos el scan:

<img src="img/snmp_metasploit_4.png" style="float:left">
<div style="clear:both"></div><br>

<a name="ntp"></a>

### 6. NTP (Network Time Protocol)

#### Breve introducción informal al protocolo NTP

El protocolo [NTP](https://es.wikipedia.org/wiki/Network_Time_Protocol) (Network Time Protocol) se utiliza para sincronizar los relojes de los equipos o dispositivos informáticos. Es decir, es el protocolo encargado de mantener los relojes de los equipos conectados a internet en hora. NTP utiliza el puerto 123 del protocolo UDP. Se pueden lanzar consultas a un servidor NTP para recopilar información como la versión del servicio o el sistema operativo. Para ampliar información sobre el protocolo NTP, podemos consultar el [RFC 5905](https://tools.ietf.org/html/rfc5905).

#### Consultando información con ntpq

Primero verificamos que el servicio NTP está activo usando nmap:

<img src="img/ntp_nmap_1.png" style="float:left">
<div style="clear:both"></div><br>

Ahora usaremos el programa [ntpq](http://doc.ntp.org/4.1.0/ntpq.htm) (ntp query) para realizar consultas al servidor NTP para obtener información. Para usarlo, le indicamos el equipo o dispositivo al que nos queremos conectar:

<img src="img/ntpq_1.png" style="float:left">
<div style="clear:both"></div><br>

Una vez conectados, podemos teclear comandos y enumerar el servidor NTP. Uno de los primeros comandos que podemos usar para recopilar información útil es *readlist*:

<img src="img/ntpq_2.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que se esta usando un servicio NTP versión 4 sobre un sistema operativo Linux sobre un procesador de arquitectura de 64bits. Otro comando interesante es peers:

<img src="img/ntpq_3.png" style="float:left">
<div style="clear:both"></div><br>

En donde vemos la lista de servidores remotos a la que nos conectamos. El que empieza con un **\*** es que primer servidor que se ha definido en el archivo de configuración del servicio ntp:

<img src="img/ntpq_4.png" style="float:left">
<div style="clear:both"></div><br>

Para ver el resto de comandos que podemos utilizar, podemos usar el comando *help*:

<img src="img/ntpq_5.png" style="float:left">
<div style="clear:both"></div><br>

#### Consultando información con nmap

Como ocurre en muchas ocasiones, existe un script (*ntp-info.nse*) en nmap que nos va a permitir enumerar el servicio ntp de forma sencilla:

<img src="img/ntp_nmap_2.png" style="float:left">
<div style="clear:both"></div><br>

<a name="dns"></a>

### 7. DNS - Transferencias de Zona

Una vez que hemos localizado los servidores DNS en el punto anterior, trataremos de identificar equipos importantes dentro de la red objetivo.

#### Breve introducción de manera muy informal a DNS

Aunque ya hicimos la introducción informal al protocolo DNS en el tema de recopilación de información, hacemos un breve repaso. Debido a que, en general, resulta más sencillo recordar nombres que secuencias numéricas, existe un protocolo llamado DNS encargado de traducir nombres de dominio como google.com o apple.es a direcciones IP. El protocolo DNS es el medio de comunicación que utilizamos para solicitar la traducción de un nombre a su dirección IP. Las traducciones se solicitan a un servidor DNS que vendría a ser, en este ejemplo, el diccionario de traducciones donde podemos buscar el nombre y obtener la IP. Al igual que ocurre con los diccionarios que clasifican las palabras en adjetivos, sustantivos, determinantes, etc. Los servidores DNS clasifican las cada una de las entradas en su diccionario con unas letras a las que llaman registros. Algunos conocidos son:

  * **A** para solicitar la IPv4 de un nombre de dominio
  * **AAAA** para solicitar la IPv6 de un nombre de dominio
  * **CNAME** para conocer los nombres (alias) de todos los servicios bajo una misma IP
  * **PTR** para conociendo la IP preguntar por el nombre de dominio
  * **NS** para saber el servidor o los servidores DNS que continen los registros del dominio
  * **MX** para saber los servidores de correo
  * **SPF** indica que servidores de correo están autorizados para enviar emails con el dominio
  * **SOA** también llamado *Start of Authority* nos indica información útil como:
    * **primary name server** cuál es el servidor DNS responsable de ese nombre de dominio
    * **responsible mail addr** quién es el responsable de ese dominio
    * **serial number** que es un registro de tiempo que cambia cada vez que se actualiza el dominio

El protocolo DNS funciona en el puerto 53 del protocolo UDP pero, para las *transferencias de zona*, también usa el puerto 53 del protocolo TCP.

#### Breve introducción informal al concepto de zona en el protocolo DNS

Hemos comentado en el punto anterior que el servidor DNS tiene un diccionario de traducciones que contiene los nombres de dominio y sus IPs asociadas clasificadas en ciertos tipos que denomina registros. Cuando, por ejemplo, solicito traducir el dominio google.com puedo, por ejemplo, solicitar traducirlo a una dirección IPv4 (registro A) o a una dirección IPv6 (registro AAAA). Este diccionario de traducciones se conoce como base de datos DNS. Cada servidor DNS tiene una base de datos DNS correspondiente al nombre de dominio o a los nombres de dominio del que es responsable. Veamos un ejemplo. Si yo quiero saber qué servidor DNS es responsable del nombre de dominio bbc.co.uk, puedo realizar una petición al servidor dns para consultar el registro *SOA (Start Of Authority)*. Para ello, inicio el programa *nslookup*:

<img src="img/dns_soa_1.png" style="float:left">
<div style="clear:both"></div><br>

y consulto el registro *SOA* del nombre de dominio bbc.co.uk para saber qué servidor DNS es responsable de este dominio. El servidor DNS responsable del nombre de dominio se llama **primary name server**:

<img src="img/dns_soa_2.png" style="float:left">
<div style="clear:both"></div><br>

Un servidor DNS puede ser responsable de uno o varios nombres de dominio. Una vez que sabemos identificar qué servidor DNS es responsable de un nombre de dominio, vamos introducir el concepto de **zona** en el protocolo DNS. Se llama zona, en el protocolo DNS, a la información del nombre de dominio o conjunto de nombres de dominio contenida en la base de datos del servidor DNS sobre el nombre de dominio o nombres de dominios de los que es responsable. Por ejemplo, la empresa ficticia *No Tenemos respuestas* puede tener un servidor DNS que sea responsable del nombre de dominio *notenemosrespuestas.es*. Que el servidor DNS sea responsable del nombre de dominio, *notenemosrespuestas.es*, quiere decir que, cualquier equipo de internet que quiera saber la dirección IP o direcciones IPs asociadas al dominio *notenemosrespuestas.es*, tendrá que preguntar al servidor DNS responsable del dominio *notenemosrespuestas.es*. Como hemos visto, denominamos zona, en el protocolo DNS, a la información del nombre de dominio o conjunto de nombres de dominio contenida en la base de datos del servidor DNS sobre el nombre de dominio o nombres de dominios de los que es responsable. Esto quiere decir, que el servidor DNS del dominio, *notenemosrespuestas.es*, contiene la base de datos con todas las direcciones IPs asociadas a ese dominio. Como estas IPs están clasificadas por registros, podemos identificar las direcciones IP de todos los equipos importantes en el  dominio, *notenemosrespuestas.es*, como los servidores de correo. Si el dominio no es muy grande y no cuentan con servidores DNS responsables para los subdominios del dominio principal, el servidor DNS responsable del nombre de dominio *notenemosrespuestas.es*, también contendrá todas las IPs asociadas a cada uno de los subdominios.

#### Breve introducción informal a las transferencias de zona

Ahora que sabemos lo que es una **zona**, veamos qué es una transferencia de zona. Normalmente, en muchas empresas, aparte del servidor DNS responsable del nombre de dominio, se implementa un segundo servidor DNS de backup para realizar copias de la base de datos del servidor principal. En nuestro caso tendríamos el servidor DNS responsable del nombre de dominio *notenemosrespuestas.es* y un segundo servidor DNS de backup para realizar copias periódicas de la información contenida en la base de datos del servidor DNS principal. Como hemos visto, una zona, en el protocolo DNS, es la información del nombre de dominio o conjunto de nombres de dominio contenida en la base de datos del servidor DNS sobre el nombre de dominio o nombres de dominios de los que es responsable. Atendiendo a que la zona es la información contenida en la base de datos del servidor DNS, una **transferencia de zona**, es simplemente copiar (transferir) la información (*zona*) de la base de datos del servidor DNS principal responsable del dominio a otro servidor DNS que, en este caso, es un servidor de backup. Las transferencias de zona usan el puerto 53 del protocolo TCP y, si se configuran de forma insegura, permiten que cualquier persona pueda ordenar una transferencia de zona y acceder a toda información contenida en la base de datos del servidor DNS principal.

#### Transferencia de zona con nslookup

Vamos a ver como utilizar el programa *nslookup* para conectarnos a un servidor de internet configurado de forma insegura y ordenar una transferencia de zona. Para ello, vamos a utilizar el dominio [zonetransfer.me](https://digi.ninja/projects/zonetransferme.php) que está configurado para poder practicar transferencias de zona:

<img src="img/zone_transfer.png" style="float:left">
<div style="clear:both"></div><br>

Lo primero será iniciar *nslookup*:

<img src="img/zone_transfer_nslookup_1.png" style="float:left">
<div style="clear:both"></div><br>

Después realizamos una petición, a través del protocolo DNS, para consultar los servidores DNS asociados al dominio *zonetransfer.me*:

<img src="img/zone_transfer_nslookup_2.png" style="float:left">
<div style="clear:both"></div><br>

Vemos que hay dos servidores DNS:

  * nsztm2.digi.ninja
  * nsztm1.digi.ninja
  
Ahora vamos a consultar el registro *SOA* para saber cuál de los 2 es el principal:

<img src="img/zone_transfer_nslookup_3.png" style="float:left">
<div style="clear:both"></div><br>

Ahora que ya sabemos que servidor DNS es el principal (nsztm1.digi.ninja) y cuál es el secundario (nsztm2.digi.ninja). Vamos a ordenar una transferencia de zona del servidor principal (nsztm1.digi.ninja) al servidor secundario (nsztm1.digi.ninja). Es decir, vamos a copiar la información contenida en la base de datos del servidor DNS (nsztm1.digi.ninja) responsable del dominio *zonetransfer.me* a un fichero de texto. Para ello, nos conectarmos al servidor DNS principal:

<img src="img/zone_transfer_nslookup_4.png" style="float:left">
<div style="clear:both"></div><br>

Para ordenar la transferencia de zona, usamos el comando *ls -d zonetransfer.me*:

<img src="img/zone_transfer_nslookup_5.png" style="float:left">
<div style="clear:both"></div><br>

y para salvar los resultados a un fichero, podemos usar una simple redirección:

<img src="img/zone_transfer_nslookup_6.png" style="float:left">
<div style="clear:both"></div><br>

En este caso, he llamado al fichero *datos_servidor_dns_zonetransfer_me.txt*. Para ver su contenido, podemos usar el comando *type*:

<img src="img/zone_transfer_nslookup_7.png" style="float:left">
<div style="clear:both"></div><br>

#### Transferencia de zona con dig

Otro programa incluido en Kali Linux con el que podemos realizar transferencias de zona es dig. Al igual que con *nslookup*, vimos como utilizar *dig* en el capítulo de recopilación de información. Para usar dig, tenemos que saber el registro DNS que se utiliza para solicitar una transferencia de zona. Es el registro *AXFR*. Luego hacemos una transferencia de zona con dig:

<img src="img/zone_transfer_dig_1.png" style="float:left">
<div style="clear:both"></div><br>

Nuevamente si queremos guardar el contenido de la transferencia en un fichero, podemos realizar una redirección:

<img src="img/zone_transfer_dig_2.png" style="float:left">
<div style="clear:both"></div><br>

y después ver el contenido del fichero. En este caso vemos las primeras 16 líneas del fichero donde hemos guardado los datos:

<img src="img/zone_transfer_dig_3.png" style="float:left">
<div style="clear:both"></div><br>